## Specs for running

In [48]:
machine_to_run_script = 'local' # 'Sherlock', 'local'
type_of_input = 'video' # 'video' 'frame'

### Import Packages

In [41]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)
import sys
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D,Conv2D,MaxPooling2D
from keras.layers import TimeDistributed
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
#import cPickle as pickle
#from video_process_utils import *
import collections
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#from PIL import Image
from PIL import Image
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0" 
#import gc

In [42]:
def ImportImage(filepath):
    #print(filepath)
    #Check if file path exists
    img = Image.open(filepath)
    return np.array(img)

### Load Data

In [4]:
if type_of_input == 'frame':

    if machine_to_run_script == 'local':
        imagepaths_labels = pd.read_csv('./data/imagepath_gdi_10_frames.csv') # TO USE REGRESSION
    elif machine_to_run_script == 'Sherlock':
        imagepaths_labels = pd.read_csv('??????/imagepath_gdi_10_frames.csv') # TO USE REGRESSION
    #imagepaths_labels = pd.read_csv('./data/imagepath_gdi_10_frames_lnorm.csv') # TO USE CLASSIFICATION
    msk = np.random.rand(len(imagepaths_labels)) < 0.8
    train=imagepaths_labels[msk]
    validation=imagepaths_labels[~msk]
    
    train_imgs = np.array([ImportImage(img) for img in train['image_path'].values])
    train_labels = np.array([label for label in train['labels'].values])
    #train_labels = np.array([label for label in train['labels_norm'].values]) # TO USE CLASSIFICATION
    validation_imgs = np.array([ImportImage(img) for img in validation['image_path'].values])
    validation_labels = np.array([label for label in validation['labels'].values])
    #validation_labels = np.array([label for label in validation['labels_norm'].values]) # TO USE CLASSIFICATION

In [43]:
# Functions CODE FOR CREATING GROUPED TIME-SERIES INPUTS

def windows(data, size, sample_stride): # define time windows to create each training example
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += sample_stride # other common options: (size / 2)

# Used for CNN/FFCN input WITHOUT SQL pre-processing
def segment_signal(data_full, input_window_size, sample_stride, height = 224, width = 224): 
    segments_timeseries = np.empty((0, input_window_size, height, width, 3)) # define segment shape for training example input
    list_of_examples = []
    labels = np.empty((0))
    for (start, end) in windows(data_full['rownum'], input_window_size, sample_stride):   
        if(end < data_full.shape[0] and # we are not at the end of the total frames
            len(data_full['rownum'][start:end]) == input_window_size and  # not sure
            int(imagepaths_wlabels['image_path'][start][21:29])==int(imagepaths_wlabels['image_path'][end-1][21:29])):  # the end patientID = start patiendID
            # Pull ten frames, crop the images, stack them
#             print(data_full['image_path'][start:end].values)
            ten_frames = np.array([ImportImage(img) for img in data_full['image_path'][start:end].values])
            ten_frames_crops_res = ten_frames[:,16::2, 50:274, :]
            ten_frames_crops_res = ten_frames_crops_res[:,0:224, :, :]
            #print(segments_timeseries.shape)
            list_of_examples.append(ten_frames_crops_res)
#             print((np.array(list_of_examples).shape))
#             print(labels.shape)
            labels = np.append(labels,data_full['labels'][start])          
#             print(labels.shape)
#             print(start, end)
            
    return np.array(list_of_examples), labels

In [49]:
if type_of_input == 'video':

    #imagepaths_labels = pd.read_csv('./data/imagepath_gdi_10_frames.csv')
    if machine_to_run_script == 'local':
        imagepaths_wlabels = pd.read_csv('./data/imagepath_gdi_10_frames.csv') # TO USE REGRESSION
    elif machine_to_run_script == 'Sherlock':
        imagepaths_wlabels = pd.read_csv('.?????/imagepath_gdi_10_frames.csv') # TO USE REGRESSION

    videos, labels = segment_signal(imagepaths_wlabels, 10, 1, height = 224, width = 224)

    videos_normalized = (videos - videos.mean())/videos.std()
    
    msk = np.random.rand(len(videos_normalized)) < 0.8
    train_videos=videos_normalized[msk]
    train_videos_labels=labels[msk]
    validation_videos=videos_normalized[~msk]
    validation_videos_labels=labels[~msk]

In [53]:
if type_of_input == 'video':

    print('train images shape : ',train_videos.shape)
    print('train labels shape : ',train_videos_labels.shape)
    print('validation images shape : ',validation_videos.shape)
    print('validation labels shape : ',validation_videos_labels.shape)
    
    #print(train_videos[0,1,10:25,10:25,0])

    # Check if any images are zeros
    for i, video in enumerate(train_videos):
        if (np.count_nonzero(video)<1):
            print('all zeros at : ', i)
    for i, video in enumerate(validation_videos):
        if (np.count_nonzero(video)<1):
            print('all zeros at : ', i)

train images shape :  (59, 10, 224, 224, 3)
train labels shape :  (59,)
validation images shape :  (12, 10, 224, 224, 3)
validation labels shape :  (12,)


In [47]:
if type_of_input == 'frame':
#     # Resize our Images to 224 x 224 x 3
     train_imgs_crops_res = train_videos[0,:,16::2, 50:274, :]
     train_imgs_crops_res = train_imgs_crops_res[:,0:224, :, :]
#     validation_imgs_crops_res = validation_imgs[:,16::2, 50:274, :]
#     validation_imgs_crops_res = validation_imgs_crops_res[:,0:224, :, :]

NameError: name 'train_videos' is not defined

In [57]:
# Mostly unnecessary, but might as well leave it in case we switch to ACNN

def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

checkpoint_folder = "./cnn_checkpoints_gdicnn"
batch_size  = 16
num_classes = len(train_labels)
epochs = 100
epochs_drop,drop_factor = (10,0.)
#input_shape = train_imgs_crops_res[0].shape
input_shape = train_videos[0].shape
kernel_size = 8
conv_dim = 16
initial_lrate = 0.001
dropout_amount = 0.5
l2_lambda = 10**(-3.5)
reg = keras.regularizers.l2(l2_lambda)
last_layer_dim=18

In [32]:
if type_of_input == 'frame':

    # Resize our Images to 224 x 224 x 3
    train_imgs_crops_res = train_imgs[:,16::2, 50:274, :]
    train_imgs_crops_res = train_imgs_crops_res[:,0:224, :, :]
    validation_imgs_crops_res = validation_imgs[:,16::2, 50:274, :]
    validation_imgs_crops_res = validation_imgs_crops_res[:,0:224, :, :]

    # Normalize Data
    
    
    # Configure VGG Model:
    vgg16_model = keras.applications.vgg16.VGG16() # Download Model
    type(vgg16_model) #This is a Keras Functional API need to convert to sequential
    Frame_model = Sequential() #Iterate over the functional layers and add it as a stack
    for layer in vgg16_model.layers:
        Frame_model.add(layer)

    # Remove last layer of VGG:
    Frame_model.layers.pop()

    # Fix the VGG model
    for layer in Frame_model.layers: #Since the model is already trained with certain weights, we dont want to change it. Let it be the same
        layer.trainable = False

    # Add a Dense layer to VGG
    Frame_model.add(Dense(64, activation='relu')) # Add the last layer Q: Why sigmoid? Why note softmax? Or lienar for regression?                
    #Frame_model.add(Dense(18, activation='softmax')) # Add the last layer
    Frame_model.add(Dense(1, activation='linear')) # Add the last layer
    
    Frame_model.summary()
    
    # Dummy model
    
    dummy_model = Sequential()
    dummy_model.add(Conv2D(conv_dim, kernel_size=kernel_size, input_shape=input_shape, padding='valid', strides = 10))
    dummy_model.add(Activation('relu'))
    dummy_model.add(BatchNormalization())
    dummy_model.add(Flatten())
    dummy_model.add(Dense(18,activation='relu'))
    dummy_model.add(Dense(1, activation='linear'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [33]:
dummy_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 22, 22, 16)        3088      
_________________________________________________________________
activation_2 (Activation)    (None, 22, 22, 16)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 22, 22, 16)        64        
_________________________________________________________________
flatten_2 (Flatten)          (None, 7744)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 18)                139410    
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 19        
Total params: 142,581
Trainable params: 142,549
Non-trainable params: 32
_________________________________________________________________


In [54]:
# DUMMY VIDEO MODEL

if type_of_input == 'video':

    dummy_model = Sequential()
    dummy_model.add(Conv2D(conv_dim, kernel_size=kernel_size, input_shape=train_videos[0][0].shape, padding='valid', strides = 5))
    dummy_model.add(Activation('relu'))
    dummy_model.add(BatchNormalization())
    dummy_model.add(Flatten())
    dummy_model.add(Dense(18,activation='relu'))

    video_input = Input(shape=(10, 224, 224, 3)) # 10 frames that are 224x224x3 each
    video_input = Input(shape=train_videos[0].shape) # 10 frames that are 224x224x3 each
    encoded_frame_sequence = TimeDistributed(dummy_model)(video_input) # Run frame_model on each frame

    # Add LSTM                  
    encoded_video = LSTM(256)(encoded_frame_sequence)  # the output will be a vector

    # Add a linear layer for output of video model
    output = Dense(1, activation='linear')(encoded_video)

    # Configure video_model
    video_model = Model(inputs=video_input, outputs=output)   
    
    video_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 10, 224, 224, 3)   0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 18)            560738    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               281600    
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 257       
Total params: 842,595
Trainable params: 842,563
Non-trainable params: 32
_________________________________________________________________


In [ ]:
if type_of_input == 'video':

    # Configure VGG Model:
    vgg16_model = keras.applications.vgg16.VGG16() # Download Model
    type(vgg16_model) #This is a Keras Functional API need to convert to sequential
    Frame_model = Sequential() #Iterate over the functional layers and add it as a stack
    for layer in vgg16_model.layers:
        Frame_model.add(layer)

    # Remove last layer of VGG:
    Frame_model.layers.pop()

    # Fix the VGG model
    for layer in Frame_model.layers: #Since the model is already trained with certain weights, we dont want to change it. Let it be the same
        layer.trainable = False

    # Add a Dense layer to VGG
    Frame_model.add(Dense(32, activation='relu')) # Add a connected layer

    video_input = Input(shape=(10, 224, 224, 3)) # 10 frames that are 224x224x3 each
    encoded_frame_sequence = TimeDistributed(Frame_model)(video_input) # Run frame_model on each frame

    # Add LSTM                  
    encoded_video = LSTM(256)(encoded_frame_sequence)  # the output will be a vector

    # Add a linear layer for output of video model
    output = Dense(1, activation='linear')(encoded_video)

    # Configure video_model
    video_model = Model(inputs=video_input, outputs=output)   
    
    video_model.summary()

In [58]:
if type_of_input == 'video':
    # train_model = True

    if not os.path.exists(checkpoint_folder):
        os.makedirs(checkpoint_folder)

    filepath=checkpoint_folder+"/weights-{epoch:02d}.hdf5"
    # if train_model:

    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    #opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    if type_of_input == 'video':
        video_model.compile(loss='mse',metrics=['mae'],optimizer=opt)
    elif type_of_input == 'video':
        Frame_model.compile(loss='mse',metrics=['accuracy'],optimizer=opt)

    #model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])


    checkpoint = \
        ModelCheckpoint(filepath, verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    #  APURVA
    # history = Frame_model.fit(train_imgs, train_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
    #           validation_data=(validation_imgs,validation_labels),
    #           batch_size=batch_size, epochs=epochs,shuffle=True)

    # ADAM
    # history = Frame_model.fit(train_imgs_crops_res, train_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
    #           validation_data=(validation_imgs_crops_res,validation_labels),
    #           batch_size=batch_size, epochs=epochs,shuffle=True)
    history = video_model.fit(train_videos, train_videos_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(validation_videos,validation_videos_labels),
              batch_size=batch_size, epochs=epochs,shuffle=True)

Train on 59 samples, validate on 12 samples
Epoch 1/100
59/59 [==============================] - 7s 125ms/step - loss: 4377.7109 - mean_absolute_error: 65.4430 - val_loss: 4817.6270 - val_mean_absolute_error: 68.5856

Epoch 00001: val_loss improved from inf to 4817.62695, saving model to ./cnn_checkpoints_gdicnn/weights-01.hdf5
Epoch 2/100
59/59 [==============================] - 5s 86ms/step - loss: 3862.6616 - mean_absolute_error: 61.3387 - val_loss: 4349.4399 - val_mean_absolute_error: 65.0796

Epoch 00002: val_loss improved from 4817.62695 to 4349.43994, saving model to ./cnn_checkpoints_gdicnn/weights-02.hdf5
Epoch 3/100
59/59 [==============================] - 5s 85ms/step - loss: 3460.5017 - mean_absolute_error: 58.0173 - val_loss: 4038.0393 - val_mean_absolute_error: 62.6546

Epoch 00003: val_loss improved from 4349.43994 to 4038.03931, saving model to ./cnn_checkpoints_gdicnn/weights-03.hdf5
Epoch 4/100
59/59 [==============================] - 5s 85ms/step - loss: 3199.1970 - 

59/59 [==============================] - 5s 91ms/step - loss: 2421.9015 - mean_absolute_error: 48.2253 - val_loss: 3073.9717 - val_mean_absolute_error: 54.4209

Epoch 00070: val_loss did not improve from 3070.78638
Epoch 71/100
59/59 [==============================] - 6s 107ms/step - loss: 2421.9178 - mean_absolute_error: 48.2254 - val_loss: 3073.9929 - val_mean_absolute_error: 54.4211

Epoch 00071: val_loss did not improve from 3070.78638
Epoch 72/100
59/59 [==============================] - 6s 103ms/step - loss: 2421.9317 - mean_absolute_error: 48.2256 - val_loss: 3074.0149 - val_mean_absolute_error: 54.4213

Epoch 00072: val_loss did not improve from 3070.78638
Epoch 73/100
59/59 [==============================] - 7s 114ms/step - loss: 2421.9505 - mean_absolute_error: 48.2258 - val_loss: 3074.0364 - val_mean_absolute_error: 54.4215

Epoch 00073: val_loss did not improve from 3070.78638
Epoch 74/100
59/59 [==============================] - 7s 113ms/step - loss: 2421.9674 - mean_absol

In [ ]:
batch_size  = 18
dropout_amount = 0.1
checkpoint_folder = "./cnn_checkpoints_gdicnn_loss_sce"
reg = keras.regularizers.l2(l2_lambda)

In [ ]:
# from keras.utils.np_utils import to_categorical
# train_categorical_labels = to_categorical(train_labels, num_classes=None)
# validation_categorical_labels = to_categorical(validation_labels, num_classes=None)

In [ ]:
# type(train_categorical_labels)
# len(train_categorical_labels[0])

In [21]:
    print('train images shape : ',train_imgs_crops_res.shape)
    print('train labels shape : ',train_labels.shape)
    print('validation images shape : ',validation_imgs_crops_res.shape)
    print('validation labels shape : ',validation_labels.shape)
    print(train_labels[0:25])
    print(train_imgs_crops_res[0,10:25,10:25,0])

train images shape :  (311, 224, 224, 3)
train labels shape :  (311,)
validation images shape :  (85, 224, 224, 3)
validation labels shape :  (85,)
[62.14180193 62.14180193 62.14180193 62.14180193 62.14180193 62.14180193
 62.14180193 62.14180193 62.14180193 62.14180193 62.14180193 73.25523767
 73.25523767 73.25523767 73.25523767 73.25523767 73.25523767 73.25523767
 73.25523767 80.44922367 80.44922367 80.44922367 80.44922367 80.44922367
 80.44922367]
[[  0   0   0   0   0   0   0   0   0   0 188 189 190 191 191]
 [  0   0   0   0   0   0   0   0   0   0 184 184 185 185 186]
 [  0   0   0   0   0   0   0   0   0 179 179 179 180 180 180]
 [  0   0   0   0   0   0   0   0 174 174 174 174 175 175 175]
 [  0   0   0   0   0   0   0   0 170 170 170 169 169 169 169]
 [  0   0   0   0   0   0   0   0 164 164 163 163 163 163 163]
 [  0   0   0   0   0   0   0   0 158 158 157 157 156 156 156]
 [  0   0   0   0   0   0   0   0 152 152 151 150 150 150 150]
 [  0   0   0   0   0   0   0   0 146 146 

In [39]:
train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}.hdf5"
if train_model:

    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    #opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    #model.compile(loss=keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'],optimizer=opt)
    #model.compile(loss=keras.losses.categorical_crossentropy,metrics=['accuracy'],optimizer=opt)
    Frame_model.compile(loss='mse', optimizer=opt)


    checkpoint = \
        ModelCheckpoint(filepath, verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

#     history = model.fit(train_imgs, train_categorical_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
#               validation_data=(validation_imgs,validation_categorical_labels),
#               batch_size=batch_size, epochs=epochs,shuffle=True)
    history = Frame_model.fit(train_imgs_crops_res, train_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
                validation_data=(validation_imgs_crops_res,validation_labels),
                batch_size=8, epochs=100,shuffle=True)

Train on 311 samples, validate on 85 samples
Epoch 1/100
311/311 [==============================] - 204s 656ms/step - loss: 5533.0440 - val_loss: 5374.4448

Epoch 00001: val_loss improved from inf to 5374.44478, saving model to ./cnn_checkpoints_gdicnn/weights-01.hdf5
Epoch 2/100
311/311 [==============================] - 215s 690ms/step - loss: 5209.7200 - val_loss: 5052.8941

Epoch 00002: val_loss improved from 5374.44478 to 5052.89407, saving model to ./cnn_checkpoints_gdicnn/weights-02.hdf5
Epoch 3/100
311/311 [==============================] - 201s 647ms/step - loss: 4888.1017 - val_loss: 4730.0425

Epoch 00003: val_loss improved from 5052.89407 to 4730.04249, saving model to ./cnn_checkpoints_gdicnn/weights-03.hdf5
Epoch 4/100
311/311 [==============================] - 202s 651ms/step - loss: 4564.4791 - val_loss: 4402.6828

Epoch 00004: val_loss improved from 4730.04249 to 4402.68277, saving model to ./cnn_checkpoints_gdicnn/weights-04.hdf5
Epoch 5/100
311/311 [=================

Epoch 37/100
311/311 [==============================] - 209s 671ms/step - loss: 150.0710 - val_loss: 135.9316

Epoch 00037: val_loss improved from 144.22531 to 135.93158, saving model to ./cnn_checkpoints_gdicnn/weights-37.hdf5
Epoch 38/100
311/311 [==============================] - 209s 672ms/step - loss: 142.5935 - val_loss: 128.8020

Epoch 00038: val_loss improved from 135.93158 to 128.80204, saving model to ./cnn_checkpoints_gdicnn/weights-38.hdf5
Epoch 39/100
311/311 [==============================] - 210s 675ms/step - loss: 136.1913 - val_loss: 122.7038

Epoch 00039: val_loss improved from 128.80204 to 122.70380, saving model to ./cnn_checkpoints_gdicnn/weights-39.hdf5
Epoch 40/100
311/311 [==============================] - 209s 672ms/step - loss: 131.0446 - val_loss: 118.8674

Epoch 00040: val_loss improved from 122.70380 to 118.86738, saving model to ./cnn_checkpoints_gdicnn/weights-40.hdf5
Epoch 41/100
311/311 [==============================] - 209s 673ms/step - loss: 127.3390

In [36]:
if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}.hdf5"
# if train_model:

opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
if type_of_input == 'video':
    video_model.compile(loss='mse',metrics=['mae'],optimizer=opt)
elif type_of_input == 'frame':
    dummy_model.compile(loss='mse',metrics=['accuracy'],optimizer=opt)

#model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])


checkpoint = \
    ModelCheckpoint(filepath, verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

#  APURVA
# history = Frame_model.fit(train_imgs, train_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
#           validation_data=(validation_imgs,validation_labels),
#           batch_size=batch_size, epochs=epochs,shuffle=True)

# ADAM
print()
history = dummy_model.fit(train_imgs_crops_res, train_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
          validation_data=(validation_imgs_crops_res, validation_labels),
          batch_size=batch_size, epochs=50,shuffle=True)
# history = video_model.fit(train_videos, train_videos_labels,callbacks=[checkpoint,lr,TerminateOnNaN()],
#           validation_data=(validation_videos,validation_videos_labels),
#           batch_size=batch_size, epochs=epochs,shuffle=True)


Train on 311 samples, validate on 85 samples
Epoch 1/50
311/311 [==============================] - 2s 8ms/step - loss: 223.0776 - acc: 0.0000e+00 - val_loss: 122.0249 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 122.02488, saving model to ./cnn_checkpoints_gdicnn/weights-01.hdf5
Epoch 2/50
311/311 [==============================] - 2s 5ms/step - loss: 60.7833 - acc: 0.0000e+00 - val_loss: 57.5159 - val_acc: 0.0000e+00oss: 61.5926 - acc: 0.0000e+

Epoch 00002: val_loss improved from 122.02488 to 57.51592, saving model to ./cnn_checkpoints_gdicnn/weights-02.hdf5
Epoch 3/50
311/311 [==============================] - 2s 5ms/step - loss: 27.9744 - acc: 0.0000e+00 - val_loss: 42.1295 - val_acc: 0.0000e+00

Epoch 00003: val_loss improved from 57.51592 to 42.12954, saving model to ./cnn_checkpoints_gdicnn/weights-03.hdf5
Epoch 4/50
311/311 [==============================] - 1s 5ms/step - loss: 16.0859 - acc: 0.0000e+00 - val_loss: 51.2305 - val_acc: 0.0000e+00

Epoch 0000

311/311 [==============================] - 1s 5ms/step - loss: 1.4257 - acc: 0.0000e+00 - val_loss: 27.2822 - val_acc: 0.0000e+00

Epoch 00039: val_loss did not improve from 26.35696
Epoch 40/50
311/311 [==============================] - 2s 5ms/step - loss: 1.6769 - acc: 0.0000e+00 - val_loss: 29.7844 - val_acc: 0.0000e+00

Epoch 00040: val_loss did not improve from 26.35696
Epoch 41/50
311/311 [==============================] - 1s 4ms/step - loss: 0.9712 - acc: 0.0000e+00 - val_loss: 29.9410 - val_acc: 0.0000e+00

Epoch 00041: val_loss did not improve from 26.35696
Epoch 42/50
311/311 [==============================] - 2s 5ms/step - loss: 1.1028 - acc: 0.0000e+00 - val_loss: 27.6095 - val_acc: 0.0000e+00

Epoch 00042: val_loss did not improve from 26.35696
Epoch 43/50
311/311 [==============================] - 1s 4ms/step - loss: 1.0322 - acc: 0.0000e+00 - val_loss: 29.2931 - val_acc: 0.0000e+00

Epoch 00043: val_loss did not improve from 26.35696
Epoch 44/50
311/311 [================

In [59]:
Ynew = video_model.predict(validation_videos)
# show the inputs and predicted outputs
for i in range(len(validation_videos)):
    print("True=%s, Predicted=%s" % (validation_videos_labels[i], Ynew[i]))

True=77.1938400448124, Predicted=[26.953377]
True=102.7519164554348, Predicted=[26.934359]
True=102.7519164554348, Predicted=[26.940372]
True=82.21418663880205, Predicted=[26.949389]
True=82.21418663880205, Predicted=[26.951986]
True=70.95870147644602, Predicted=[26.937592]
True=74.10904526666386, Predicted=[26.965221]
True=86.9765613362632, Predicted=[26.931103]
True=76.6035011342479, Predicted=[26.933493]
True=76.34763844275375, Predicted=[26.944248]
True=73.57620273986909, Predicted=[26.948864]
True=70.71924384116333, Predicted=[26.92424]


In [37]:
Ynew = dummy_model.predict(validation_imgs_crops_res)
# show the inputs and predicted outputs
for i in range(len(validation_imgs_crops_res)):
    print("True=%s, Predicted=%s" % (validation_labels[i], Ynew[i]))

True=73.25523767440694, Predicted=[71.599556]
True=73.25523767440694, Predicted=[69.644]
True=73.25523767440694, Predicted=[72.07249]
True=80.44922366806699, Predicted=[82.363655]
True=80.44922366806699, Predicted=[83.06998]
True=80.44922366806699, Predicted=[80.53925]
True=80.44922366806699, Predicted=[73.81894]
True=84.74102379560351, Predicted=[84.90053]
True=84.74102379560351, Predicted=[80.85953]
True=71.6790492593985, Predicted=[71.89114]
True=77.1938400448124, Predicted=[73.6715]
True=77.1938400448124, Predicted=[77.74444]
True=77.1938400448124, Predicted=[80.328606]
True=77.1938400448124, Predicted=[74.739876]
True=75.07288766535675, Predicted=[76.482605]
True=75.07288766535675, Predicted=[71.59193]
True=75.07288766535675, Predicted=[84.38763]
True=62.133284639290245, Predicted=[63.93198]
True=62.133284639290245, Predicted=[62.06683]
True=61.39248667646541, Predicted=[60.929028]
True=61.39248667646541, Predicted=[63.087093]
True=61.39248667646541, Predicted=[56.62547]
True=61.3

## Scratch

In [ ]:
# model = Sequential()
# model.add(Conv2D(conv_dim, kernel_size=kernel_size, input_shape=input_shape, padding='same'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=2))
# model.add(Dropout(dropout_amount))
# model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=2))
# model.add(Dropout(dropout_amount))
# model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(conv_dim,kernel_size=kernel_size,padding='same',kernel_regularizer=reg))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=3))
# model.add(Dropout(dropout_amount))
# model.add(Flatten())
# model.add(Dense(last_layer_dim,activation='relu'))
# model.add(Dense(18, activation='softmax'))


# from skimage import data, color
# from skimage.transform import rescale, resize, downscale_local_mean
# arr = train_imgs[0]
# #arr_resize = scipy.misc.imresize(arr,(224,224,3))
# image_resized = resize(arr, (224,224,3), anti_aliasing=false)

# print('train images shape : ',train_imgs.shape)
# print('validation images shape : ',validation_imgs.shape)

# # RESHAPE FOR VGG16
# train_imgs_crops_res = train_imgs[:,16::2, 50:274, :]
# train_imgs_crops_res = train_imgs_crops_res[:,0:224, :, :]
# validation_imgs_crops_res = validation_imgs[:,16::2, 50:274, :]
# validation_imgs_crops_res = validation_imgs_crops_res[:,0:224, :, :]

# print(train_imgs_crops_res.shape)
# print(validation_imgs_crops_res.shape)

# print(train_videos.shape)
# print(train_videos_labels.shape)
# # validation_videos=videos[~msk]
# validation_videos_labels=labels[~msk]